# HW 6

Muyuan Zhang

u1430770

07/23/2023

## Step 1: Data Acquisition + Cleanup

In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('fonts/ARIAL.csv')
df.drop(columns=['font', 'fontVariant', 'strength', 'italic', 'orientation',
'm_top', 'm_left', 'originalH', 'originalW', 'h', 'w'], inplace=True)

# takes in one of these types of dataframe and returns 2 numpy arrays:
# Xs which is a #samples x 20 x 20 array containing the pixel values, and
# Ys which is a #samples x 1 array containing the ascii value for each character
def parse_df(df):
    Xs = df.drop(columns='m_label').to_numpy(dtype=np.float64)
    Xs = np.array([x.reshape(20, 20) for x in Xs], dtype=np.float64)/255
    Xs = np.reshape(Xs, (-1, 1, 20, 20))

    # assign each character a smaller index value
    index, Ys = np.unique(df["m_label"].to_numpy(), return_inverse=True)
    
    return Xs, Ys, index

Xs, Ys, index = parse_df(df)

## Step 2: Build a PyTorch Network

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()

        self.convolution1 = nn.Conv2d(1, 8, 3)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.dense1 = nn.Linear(576, 4000)

        self.convolution2 = nn.Conv2d(8, 64, 3)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.dense2 = nn.Linear(4000, 3209)

    def forward(self, x):
        x = self.pool1(F.relu(self.convolution1(x)))
        x = self.pool2(F.relu(self.convolution2(x)))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.dense1(x))
        x = self.dense2(x)
        
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1

        for s in size:
            num_features *= s

        return num_features

network = Network()

In [ ]:
import torch
import torch.optim as optim

def train(model, epochs, data, labels):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam( model.parameters(), lr=1e-4)

    model.float()

    for epoch in range(epochs):
        running_loss = 0.0
        optimizer.zero_grad()
        outputs = model(data.float())
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print("Training completed.")

## Step 3: Exploration and Evaluation

+ Evaluate the network using cross validation (splitting data into training/testing). What is its accuracy?

In [ ]:
from sklearn.model_selection import train_test_split

def evaluate(model, data, labels):
    #load some test data
    correct = 0
    total = 0

    with torch.no_grad():
        outputs = model(data.float())
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network: %d%%' % (100 * correct / total))

data = torch.from_numpy(Xs)
labels = torch.from_numpy(Ys)
data, labels = data.to(), labels.to()
X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.5, shuffle=True)

train(network, 10, X_train, y_train)
evaluate(network, X_test, y_test)

+ Create and train a different network topology (add more convolution layers, experiment with normalization (batch normalization or dropout), explore other types/sizes of layer). Try to find a topology that works better than the one described above.

In [ ]:
class Network2(nn.Module):
    def __init__(self):
        super(Network2, self).__init__()

        self.convolution1 = nn.Conv2d(1, 8, 2)
        self.pooling1 = nn.MaxPool2d(2, 2)
        self.dense1 = nn.Linear(128, 5000)

        self.convolution2 = nn.Conv2d(8, 64, 2)
        self.pooling2 = nn.MaxPool2d(2, 2)
        self.dense2 = nn.Linear(5000, 3098)

        self.convolution3 = nn.Conv2d(64, 128, 2)
        self.pooling3 = nn.MaxPool2d(2, 2)

        self.dropout = nn.Dropout()

    def forward(self, x):
        x = self.pooling1(F.relu(self.convolution1(x)))
        x = self.dropout(x)
        x = self.pooling2(F.relu(self.convolution2(x)))
        x = self.dropout(x)
        x = self.pooling3(F.relu(self.convolution3(x)))

        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.dense1(x))
        x = self.dense2(x)

        return x

    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1

        for s in size:
            num_features *= s

        return num_features

network2 = Network2()
data = torch.from_numpy(Xs)
labels = torch.from_numpy(Ys)
data, labels = data.to(), labels.to()

train(network2, 10, data, labels)
evaluate(network2, data, labels)

+ Test the accuracy of your network with character inputs from a DIFFERENT font set. How does it perform?

In [ ]:
courier_df = pd.read_csv('fonts/COURIER.csv')
courier_df.drop(columns=['font', 'fontVariant', 'strength', 'italic', 'orientation', 'm_top', 'm_left', 'originalH', 'originalW', 'h', 'w'], inplace=True)
courier_x, courier_y, keys = parse_df(courier_df)
courier_inputs, courier_labels = torch.from_numpy(courier_x).to(), torch.from_numpy(courier_y).to()

evaluate(network2, courier_inputs, courier_labels)

+ Train your best network on inputs from the data from at least 2 different fonts. How does your accuracy compare to the 1-font case? What accuracy do you see when testing with inputs from a font you didn't train on?

In [ ]:
cambria_df = pd.read_csv('fonts/CAMBRIA.csv')
bernard_df = pd.read_csv('fonts/BERNARD.csv')

cambria_df.drop(columns=['font', 'fontVariant', 'strength', 'italic', 'orientation', 'm_top', 'm_left', 'originalH', 'originalW', 'h', 'w'], inplace=True)
bernard_df.drop(columns=['font', 'fontVariant', 'strength', 'italic', 'orientation', 'm_top', 'm_left', 'originalH', 'originalW', 'h', 'w'], inplace=True)

cambria_x, cambria_y, keys = parse_df(cambria_df)
bernard_x, bernard_y, keys = parse_df(bernard_df)

cambria_inputs, cambria_labels = torch.from_numpy(cambria_x).to(), torch.from_numpy(cambria_y).to()
bernard_inputs, bernard_labels = torch.from_numpy(bernard_x).to(), torch.from_numpy(bernard_y).to()

evaluate(network2, cambria_inputs, cambria_labels)
evaluate(network2, bernard_inputs, bernard_labels)

## Step 4: Denoising

In [ ]:
def addNoise(x):
    return max(min(x + np.random.normal() / 5, 1), 0)

X_noisy = []

for value in Xs:
    font_value = value[0]
    noisy_font_value = np.vectorize(addNoise)(font_value)
    X_noisy.append(noisy_font_value)

for i in range(500, 510):
    image = Image.fromarray(np.uint8(X_noisy[i] * 255), 'L')
    display(image)

X_noisy = torch.from_numpy(np.reshape(np.array(X_noisy), (-1, 1, 20, 20)))



In [ ]:
class CnnDenoiseNet(nn.Module):
    def __init__(self):
        super(CnnDenoiseNet, self).__init__()

        self.encodedSize = 32

        self.c1Output = 8
        self.c2Output = 8

        self.conv1 = nn.Conv2d(1, self.c1Output, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(self.c1Output, self.c2Output, 3, padding=1)

        self.downscalingSize = 20 // 4
        self.flatteningSize = self.downscalingSize * self.downscalingSize * self.c2Output

        self.fc1 = nn.Linear(self.flatteningSize, 64)
        self.fc2 = nn.Linear(64, self.encodedSize)

        self.fc3 = nn.Linear(self.encodedSize, 64)
        self.fc4 = nn.Linear(64, self.flatteningSize)


        self.upSample = nn.Upsample(scale_factor=2, mode='bilinear')
        self.cv3 = nn.Conv2d(self.c2Output, self.c1Output, 3, padding=1)
        self.cv4 = nn.Conv2d(self.c1Output, 1, 3, padding=1)

        self.double()


    def compress(self, x):
        x = self.conv1(x)
        x = F.relu(self.pool(x))
        x = self.conv2(x)
        x = F.relu(self.pool(x))
        x = x.view(-1, self.flatteningSize)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x

    def decompress(self, x):
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = x.view(-1, self.c2Output, self.downscalingSize, self.downscalingSize)
        x = self.upSample(x)
        x = F.relu(self.cv3(x))
        x = self.cv4(self.upSample(x))

        return x

    def forward(self, x):
        x = self.compress(x)
        x = self.decompress(x)

        return x

denoise_network = CnnDenoiseNet()

In [ ]:
import matplotlib.pyplot as plt

def plotComparisons(model):

    plt.figure( figsize=(20, 20) )

    testing_loader = torch.utils.data.DataLoader( X_noisy, batch_size=8, shuffle=True, num_workers=0 )

    for i, image in enumerate(testing_loader):
        if i >= 8:
            break
        images = image[0]

        with torch.no_grad():
            denoised = model(images.to(torch.float32).to(device))
            for j in range( len(images) ):

                # Original
                ax = plt.subplot(16, 8, i*16 + j + 1)
                plt.imshow(images[j].cpu().reshape(20,20), cmap="Greys", interpolation=None)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

                # Denoised
                ax = plt.subplot(16, 8, i*16 + j + 1 + 8)
                plt.imshow(denoised[j].cpu().reshape(20,20), cmap="Greys", interpolation=None)
                ax.get_xaxis().set_visible(False)
                ax.get_yaxis().set_visible(False)

plotComparisons(denoise_network)
torch.cuda.empty_cache()